In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math
import time 

num_qubits = 4
num_layers = 2

dev = qml.device("default.qubit", wires=num_qubits)

# quantum circuit functions
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

def layer(W):

    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# preparaing data
df_train = pd.read_csv('train.csv')

df_train['Pclass'] = df_train['Pclass'].astype(str)

df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)

print(df_train)

# I will fill missings with the median
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived'])

X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# setting init params
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer(0.125)
num_it = 70
batch_size = math.floor(len(X_train)/num_it)

# Start timing the training process
start_time = time.time()

weights = weights_init
bias = bias_init
for it in range(num_it):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )

# End timing the training process
end_time = time.time()
elapsed_time = end_time - start_time

print("Total training time: {:.2f} seconds".format(elapsed_time))

X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

accuracy_score(Y_test, predictions)
precision_score(Y_test, predictions)
recall_score(Y_test, predictions)
f1_score(Y_test, predictions, average='macro')

     PassengerId  Survived Pclass  \
0              1         0      3   
1              2         1      1   
2              3         1      3   
3              4         1      1   
4              5         0      3   
..           ...       ...    ...   
886          887         0      2   
887          888         1      1   
888          889         0      3   
889          890         1      1   
890          891         0      3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ...    ...   
88

0.7712374581939799

In [27]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math
import time 

num_qubits = 4
num_layers = 2

dev = qml.device("default.qubit", wires=num_qubits)

# quantum circuit functions
def statepreparation(x):
    #qml.BasisEmbedding(x, wires=range(0, num_qubits))
    qml.AngleEmbedding(x, wires=range(num_qubits))

def layer(W):

    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# preparaing data
#df_train = pd.read_csv('train.csv')
df_train = pd.read_csv('Datasets/adult/adult_test_int.csv')

# df_train['Pclass'] = df_train['Pclass'].astype(str)

#df_train['workclass'] = df_train['workclass'].astype(str)
#df_train['race'] = df_train['race'].astype(str)
#df_train['sex'] = df_train['sex'].astype(str)

#df_train = pd.concat([df_train, pd.get_dummies(df_train[['workclass', 'race', 'sex']])], axis=1)

#print(df_train)

# I will fill missings with the median
#df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

#df_train['is_child'] = df_train['age'].map(lambda x: 1 if x < 12 else 0)
#cols_model = ['is_child', 'workclass_2', 'race_4', 'sex_1']
cols_model = ['age', 'workclass', 'race', 'sex']

#X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived'])
X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model],
                                                    df_train['over50K'],
                                                    test_size=0.10,
                                                    random_state=42,
                                                    stratify=df_train['over50K'])

X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

# setting init params
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer(0.125)
num_it = 70
batch_size = math.floor(len(X_train)/num_it)

# Start timing the training process
start_time = time.time()

weights = weights_init
bias = bias_init
for it in range(num_it):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )

# End timing the training process
end_time = time.time()
elapsed_time = end_time - start_time

print("Total training time: {:.2f} seconds".format(elapsed_time))

X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

accuracy_score(Y_test, predictions)
precision_score(Y_test, predictions, zero_division=1)
recall_score(Y_test, predictions)
f1_score(Y_test, predictions, average='macro')

Iter:     1 | Cost: 0.9468969 | Accuracy: 0.5972165 
Iter:     2 | Cost: 0.8746061 | Accuracy: 0.6582071 
Iter:     3 | Cost: 0.8307673 | Accuracy: 0.7187884 


KeyboardInterrupt: 

In [28]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import pandas as pd
import math
import time

# -------------------------------
# Data Preparation
# -------------------------------
df_train = pd.read_csv('Datasets/adult/adult_test_int.csv')

# One-hot encode categorical columns
df_train = pd.get_dummies(df_train)

# Normalize all columns
scaler = MinMaxScaler()
df_train[df_train.columns] = scaler.fit_transform(df_train)

# Prepare features and target
X = df_train.drop(columns=['over50K'])  # Use all columns except the target
y = df_train['over50K']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42, stratify=y
)

# Convert to pennylane-compatible numpy arrays
X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)

X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

# -------------------------------
# Quantum Circuit Configuration
# -------------------------------
num_qubits = X_train.shape[1]  # Number of qubits matches the number of features
num_layers = 2

dev = qml.device("default.qubit", wires=num_qubits)

def statepreparation(x):
    """Embed the features into the quantum state using AngleEmbedding."""
    qml.AngleEmbedding(x, wires=range(num_qubits), rotation='Y')

def layer(W):
    """Apply a layer of rotations followed by CNOTs in a chain structure."""
    for i in range(num_qubits):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
    for i in range(num_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    """Define the quantum circuit."""
    statepreparation(x)
    for W in weights:
        layer(W)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# -------------------------------
# Loss and Cost Functions
# -------------------------------
def square_loss(labels, predictions):
    loss = sum((l - p) ** 2 for l, p in zip(labels, predictions)) / len(labels)
    return loss

def accuracy_metric(labels, predictions):
    correct = sum(1 for l, p in zip(labels, predictions) if abs(l - p) < 1e-5)
    return correct / len(labels)

def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# -------------------------------
# Training
# -------------------------------
np.random.seed(0)
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = AdamOptimizer(0.125)
num_it = 70
batch_size = max(1, math.floor(len(X_train) / num_it))

weights = weights_init
bias = bias_init

start_time = time.time()
for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    
    # Update weights and bias
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)
    
    # Compute training accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy_metric(Y_train, predictions)
    
    print(f"Iter: {it + 1:5d} | Cost: {cost(weights, bias, X_train, Y_train):0.7f} | Accuracy: {acc:0.7f}")

end_time = time.time()
print(f"Total training time: {elapsed_time:.2f} seconds")

# -------------------------------
# Evaluation
# -------------------------------
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

print("Accuracy Score:", accuracy_score(Y_test, predictions))
print("Precision Score:", precision_score(Y_test, predictions, zero_division=1))
print("Recall Score:", recall_score(Y_test, predictions))
print("F1 Score:", f1_score(Y_test, predictions, average='macro'))


Iter:     1 | Cost: 2.0124435 | Accuracy: 0.2570610
Iter:     2 | Cost: 1.5278867 | Accuracy: 0.2656570
Iter:     3 | Cost: 1.1252381 | Accuracy: 0.4457634
Iter:     4 | Cost: 0.8645769 | Accuracy: 0.6692591
Iter:     5 | Cost: 0.7869944 | Accuracy: 0.7429390
Iter:     6 | Cost: 0.8522083 | Accuracy: 0.7429390
Iter:     7 | Cost: 0.9449780 | Accuracy: 0.7429390
Iter:     8 | Cost: 0.9755166 | Accuracy: 0.7429390
Iter:     9 | Cost: 0.9299182 | Accuracy: 0.7429390
Iter:    10 | Cost: 0.8540516 | Accuracy: 0.7429390
Iter:    11 | Cost: 0.7897069 | Accuracy: 0.7429390
Iter:    12 | Cost: 0.7597606 | Accuracy: 0.7429390
Iter:    13 | Cost: 0.7510826 | Accuracy: 0.7429390
Iter:    14 | Cost: 0.7529657 | Accuracy: 0.7429390
Iter:    15 | Cost: 0.7571072 | Accuracy: 0.7429390
Iter:    16 | Cost: 0.7620149 | Accuracy: 0.7429390
Iter:    17 | Cost: 0.7650518 | Accuracy: 0.7429390
Iter:    18 | Cost: 0.7681715 | Accuracy: 0.7429390
Iter:    19 | Cost: 0.7708597 | Accuracy: 0.7429390
Iter:    20 